# OpenPoseNetの実装

# 4.3 目標

## OpenPose全体
1.	OpenPoseのネットワーク構造をモジュール単位で理解する
2.	OpenPoseを構成する各モジュールのサブネットワーク構成を理解する
3.	OpenPoseのネットワーククラスを実装できるようになる

## 各モジュール
1.	Featureモジュールのサブネットワーク構成を理解する
2.	Featureモジュールを実装できるようになる
3.	Stageモジュールのblockを構成するサブネットワークを理解する
4.	Stageモジュールを実装できるようになる

## Library

In [1]:
import torch
import torch.nn as nn
from torch.nn import init
import torchvision

In [65]:
# OpenPoseNetクラスの実装
# コンストラクタで各モジュールを生成し、forward関数を定義する

class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet, self).__init__()
    
        # Featureモジュール
        self.model0 = OpenPose_Feature()

        # Stageモジュール
        # PAFs(Part Affinity Fields)側
        self.model1_1 = make_OpenPose_block('block1_1')
        self.model2_1 = make_OpenPose_block('block2_1')
        self.model3_1 = make_OpenPose_block('block3_1')
        self.model4_1 = make_OpenPose_block('block4_1')
        self.model5_1 = make_OpenPose_block('block5_1')
        self.model6_1 = make_OpenPose_block('block6_1')

        # confidence heatmap側
        self.model1_2 = make_OpenPose_block('block1_2')
        self.model2_2 = make_OpenPose_block('block2_2')
        self.model3_2 = make_OpenPose_block('block3_2')
        self.model4_2 = make_OpenPose_block('block4_2')
        self.model5_2 = make_OpenPose_block('block5_2')
        self.model6_2 = make_OpenPose_block('block6_2')

    def forward(self, x):   # ここの綴り間違っていてNotImplementedErrorがでていた
        """
        順伝播の定義
        
        Returuns
        ------------
        out6_1 : torch.Size([minibatch, 38, 46, 46])
        out6_2 : torch.Size([minibatch, 19, 46, 46])
        saved_for_loss : [out1_1, out1_2, out2_1, ... , out6_2]
        """
        
        # Featureモジュール
        out1 = self.model0(x)
        
        # Stage1
        out1_1 = self.model1_1(out1)
        out1_2 = self.model1_2(out1)
        
        # Stage2
        out2 = torch.cat([out1, out1_1, out1_2], 1)  #次元１で結合　[batch_num, channel, height, width]
        out2_1 = self.model2_1(out2)
        out2_2 = self.model2_2(out2)
        
        # Stage3
        out3 = torch.cat([out1, out2_1, out2_2], 1)
        out3_1 = self.model3_1(out3)
        out3_2 = self.model3_2(out3)
        
        # Stage4
        out4 = torch.cat([out1, out3_1, out3_2], 1)
        out4_1 = self.model4_1(out4)
        out4_2 = self.model4_2(out4)
        
        # Stage5
        out5 = torch.cat([out1, out4_1, out4_2], 1)
        out5_1 = self.model5_1(out5)
        out5_2 = self.model5_2(out5)
        
        # Stage6
        out6 = torch.cat([out1, out5_1, out5_2], 1)
        out6_1 = self.model6_1(out6)
        out6_2 = self.model6_2(out6)
        
        # 損失の計算用に各Stageの結果を格納  よく分かってない
        saved_for_loss = []
        saved_for_loss.append(out1_1)  # PAFs側
        saved_for_loss.append(out1_2)  # confidence heatmap側
        saved_for_loss.append(out2_1)  
        saved_for_loss.append(out2_2)
        saved_for_loss.append(out3_1)
        saved_for_loss.append(out3_2)
        saved_for_loss.append(out4_1)
        saved_for_loss.append(out4_2)
        saved_for_loss.append(out5_1)
        saved_for_loss.append(out5_2)
        saved_for_loss.append(out6_1)
        saved_for_loss.append(out6_2)
        
        # モデルの最終的な出力out6_1(PAFs), out6_2(confidence heatmap)と
        # 損失関数計算用に各ステージでのPAFｓとheatmapを格納したsaved_for_lossを出力
        # 中身を見ればout6_1, out6_2は2回返されていることになる
        
        return (out6_1, out6_2), saved_for_loss

## Featureモジュール

In [66]:
class OpenPose_Feature(nn.Module):
    """
    OpenPoseのFeatureモジュール
    今回はvgg19の0-22そうまでを取り出す
    コンストラクタで層を定義してインスタンス変数として保持する
    forwardで順伝播して結果を返す
    """
    def __init__(self):
        super(OpenPose_Feature, self).__init__()
        
        # vgg-19の最初の10この畳込みを使用
        vgg19 = torchvision.models.vgg19(pretrained=True)
        
        # 最後に(conv2d+ReLU)を2つ追加
        model = {}  # block0とblock1をまとめて管理するリスト
        model['block0'] = vgg19.features[0:23]  # VGGの最初の10の畳み込み層（の次のReLU）まで
        model['block0'].add_module('23', torch.nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1))
        model['block0'].add_module('24', torch.nn.ReLU(inplace=True))
        model['block0'].add_module('25', torch.nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1))
        model['block0'].add_module('26', torch.nn.ReLU(inplace=True))
        
        self.model = model['block0']
        
    def forward(self, x):
        outputs = self.model(x)
        return outputs

In [67]:
vgg19 = torchvision.models.vgg19(pretrained=True)
vgg19

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [68]:
a = vgg19.features[:23]
a.add_module('23', torch.nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1))
print(type(a))
# Sequentialにはadd_moduleで層を追加できる
a

<class 'torch.nn.modules.container.Sequential'>


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

## Stageモジュール

In [69]:
def make_OpenPose_block(block_name):
    """
    コンフィグレーション変数からOpenPoseStageモジュールのblockを作成
    nn.Moduleではなくnn.Sequentialにする　　なんで？
    """
    
    # 1. コンフィグレーションの辞書変数blocksを作成し、ネットワークを生成する
    # 最初に全パターンの辞書を用意し引数block_nameのみを生成する
    blocks = {}
    
    # stage1
    blocks['block1_1'] = [{'conv5_1_CPM_L1' : [128, 128, 3, 1, 1]},
                                        {'conv5_2_CPM_L1' : [128, 128, 3, 1, 1]},
                                        {'conv5_3_CPM_L1' : [128, 128, 3, 1, 1]},
                                        {'conv5_4_CPM_L1' : [128, 512, 1, 1, 0]},
                                        {'conv5_5_CPM_L1' : [512, 38, 1, 1, 0]}]
    
    blocks['block1_2'] = [{'conv5_1_CPM_L2' : [128, 128, 3, 1, 1]},
                                        {'conv5_2_CPM_L2' : [128, 128, 3, 1, 1]},
                                        {'conv5_3_CPM_L2' : [128, 128, 3, 1, 1]},
                                        {'conv5_4_CPM_L2' : [128, 512, 1, 1, 0]},
                                        {'conv5_5_CPM_L2' : [512, 19, 1, 1, 0]}]
    
    # stage2-6
    for i in range(2, 7):
        blocks['block%d_1' % i] = [
            {'Mconv1_stage%d_L1' % i : [185, 128, 7, 1, 3]},
            {'Mconv2_stage%d_L1' % i : [128, 128, 7, 1, 3]},
            {'Mconv3_stage%d_L1' % i : [128, 128, 7, 1, 3]},
            {'Mconv4_stage%d_L1' % i : [128, 128, 7, 1, 3]},
            {'Mconv5_stage%d_L1' % i : [128, 128, 7, 1, 3]},
            {'Mconv6_stage%d_L1' % i : [128, 128, 1, 1, 0]},
            {'Mconv7_stage%d_L1' % i : [128, 38, 1, 1, 0]}
        ]
        blocks['block%d_2' % i] = [
            {'Mconv1_stage%d_L2' % i : [185, 128, 7, 1, 3]},
            {'Mconv2_stage%d_L2' % i : [128, 128, 7, 1, 3]},
            {'Mconv3_stage%d_L2' % i : [128, 128, 7, 1, 3]},
            {'Mconv4_stage%d_L2' % i : [128, 128, 7, 1, 3]},
            {'Mconv5_stage%d_L2' % i : [128, 128, 7, 1, 3]},
            {'Mconv6_stage%d_L2' % i : [128, 128, 1, 1, 0]},
            {'Mconv7_stage%d_L2' % i : [128, 19, 1, 1, 0]}
        ]
        
    # 引数block_nameのコンフィグレーション辞書を取り出す
    cfg_dict = blocks[block_name]
    
    # 2.　コンフィグレーション内容をリスト変数layersに格納
    layers = []
    
    # 0番目から最後の層までを作成
    for i in range(len(cfg_dict)):
        for k, v in cfg_dict[i].items():  # key: 層の名前, value: config?
            if 'pool' in k:
                layers += [nn.MaxPooling2d(kernel_size=v[0], stride=v[1], padding=v[2])]
            else:
                conv2d = nn.Conv2d(in_channels=v[0], out_channels=v[1], kernel_size=v[2], stride=v[3], padding=v[4])
                layers += [conv2d, nn.ReLU(inplace=True)]
                
    # 3. layersをSequentiaにする
    # ただし、最後にReLUはいらないのでその手前まで
    net = nn.Sequential(*layers[:-1])
    
    # 初期化関数の設定をして畳み込み層の重みを初期化する
    # 初期化関数 init.normal_(tensor, mean=0.0, std=1.0)  正規分布で初期化する
    # 初期化関数 init.constant_(tensor, val)  valで初期化する
    def _initialize_weights_norm(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    init.constant_(m.bias, 0.0)
                    
    net.apply(_initialize_weights_norm)
    
    return net

In [70]:
# stageモジュールの動作確認
net = OpenPoseNet()
net.train()  # 訓練モードへ

# ダミーデータの作成
batch_size = 2
dummy_img = torch.rand(batch_size, 3, 368, 368)

# 計算
outputs = net(dummy_img)
print(outputs)

((tensor([[[[ 2.8397e-05,  4.6908e-05,  3.1456e-05,  ...,  5.2471e-05,
            7.4813e-05,  2.1722e-05],
          [ 4.2686e-05,  8.8065e-05,  8.8867e-05,  ...,  5.5875e-05,
            7.3833e-05,  5.1843e-05],
          [ 4.7339e-05,  7.6238e-05,  8.1362e-05,  ...,  6.4683e-05,
            8.0689e-05,  6.5621e-05],
          ...,
          [ 8.5527e-06,  3.1697e-05,  1.7968e-05,  ...,  1.0442e-05,
            1.2439e-05,  5.2723e-05],
          [ 2.8877e-05,  3.0968e-06, -1.5241e-05,  ..., -4.8340e-06,
           -8.2196e-06,  2.8881e-05],
          [ 1.9624e-05,  2.5232e-05,  2.0443e-05,  ..., -2.0123e-05,
           -8.1424e-06,  1.7108e-05]],

         [[-9.7441e-06, -8.9028e-06,  1.3478e-05,  ...,  1.0205e-05,
            1.7266e-05, -2.8113e-06],
          [-1.6563e-05, -3.0057e-05,  3.4888e-05,  ...,  1.3818e-05,
            7.3930e-06,  1.2630e-05],
          [-6.3479e-06, -2.3552e-05,  2.5929e-05,  ...,  2.5680e-05,
           -1.8834e-05, -2.6633e-06],
          ...,
   